In [ ]:
import io, os, sys, collections
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def bfs(C, F, l, src, snk) :
  V = len(C)
  Q = collections.deque()
  Q.append(src)
  l[src] = 0
  while Q :
    u = Q.popleft()
    for v in range(V) :
      if l[v] == -1 and C[u][v] - F[u][v] > 0 :
        l[v] = l[u] + 1
        Q.append(v)
  return l[snk] != -1

def dfs(C, F, W, l, u, snk, cur = 1 << 63) :
  if u == snk : return cur
  V = len(C)
  while W[u] < V :
    v = W[u]
    if l[v] == l[u] + 1 and C[u][v] > F[u][v] :
      A = min(cur, C[u][v] - F[u][v])
      tmp = dfs(C, F, W, l, v, snk, A)
      if tmp > 0 :
        F[u][v] += tmp
        F[v][u] -= tmp
        return tmp
    W[u] += 1
  return 0

def dinic(C, src, snk) :
  V = len(C)
  F = [[0] * V for _ in range(V)]
  ret = 0

  while True :
    l = [-1] * V
    W = [0] * V
    if not bfs(C, F, l, src, snk) : break
    while True:
      tmp = dfs(C, F, W, l, src, snk)
      if tmp == 0 : break
      ret += tmp
  return ret

#K=75, H=72, .=46, #=35
def sol():
  N, M = map(int, input().split())
  G = [[*input().rstrip()] for _ in range(N)]
  V = N * M * 2 + 2
  C = [[0] * V for _ in range(V)]
  src = V - 2
  snk = V - 1

  NM = N * M
  idx = lambda x, y : x * M + y
  for i in range(N) :
    for j in range(M) :
      if G[i][j] == 35 : continue
      for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)) :
        nx = i + dx
        ny = j + dy
        if not( 0 <= nx < N and 0 <= ny < M) : continue
        if G[nx][ny] == 35 : continue

        C[NM+idx(i, j)][idx(nx, ny)] = 1 #out -> (another)in

      C[idx(i, j)][NM+idx(i, j)] = 1 #in - > out
      if G[i][j] == 75 :
        KPos = (i, j)
        C[src][idx(i, j)] = 1 << 30
        C[idx(i, j)][NM+idx(i, j)] = 1 << 30
      elif G[i][j] == 72 :
        C[idx(i, j)][NM+idx(i, j)] = 1 << 30
        C[NM+idx(i, j)][snk] = 1 << 30

  for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)) :
    nx = KPos[0] + dx
    ny = KPos[1] + dy
    if not( 0 <= nx < N and 0 <= ny < M) : continue
    if G[nx][ny] == 72 : 
      return print(-1)
  
  print(dinic(C, src, snk))
sol()

- MLE. 아이디어는 맞는 것 같은데 그래프의 정점마다 유량그래프의 정점 하나씩을 두는건 아닌 것 같다.

In [ ]:
import io, os, sys
sys.setrecursionlimit(10**6)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = float("inf")
class Dinic:
  def __init__(self, n):
    self.lvl = [0] * n
    self.ptr = [0] * n
    self.q = [0] * n
    self.adj = [[] for _ in range(n)]

  def add_edge(self, a, b, c, rcap=0):
    self.adj[a].append([b, len(self.adj[b]), c, 0])
    self.adj[b].append([a, len(self.adj[a]) - 1, rcap, 0])

  def dfs(self, v, t, f):
    if v == t or not f:
      return f

    for i in range(self.ptr[v], len(self.adj[v])):
      e = self.adj[v][i]
      if self.lvl[e[0]] == self.lvl[v] + 1:
        p = self.dfs(e[0], t, min(f, e[2] - e[3]))
        if p:
          self.adj[v][i][3] += p
          self.adj[e[0]][e[1]][3] -= p
          return p
      self.ptr[v] += 1

    return 0

  def calc(self, s, t):
    flow, self.q[0] = 0, s
    for l in range(31):
      while True:
        self.lvl, self.ptr = [0] * len(self.q), [0] * len(self.q)
        qi, qe, self.lvl[s] = 0, 1, 1
        while qi < qe and not self.lvl[t]:
          v = self.q[qi]
          qi += 1
          for e in self.adj[v]:
            if not self.lvl[e[0]] and (e[2] - e[3]) >> (30 - l):
              self.q[qe] = e[0]
              qe += 1
              self.lvl[e[0]] = self.lvl[v] + 1

        p = self.dfs(s, t, INF)
        while p:
          flow += p
          p = self.dfs(s, t, INF)

        if not self.lvl[t]:
          break

    return flow

#K=75, H=72, .=46, #=35
def sol():
  N, M = map(int, input().split())
  G = [[*input().rstrip()] for _ in range(N)]
  if N == M == 1: return print(-1)
  V = N * M * 2 + 2
  C = Dinic(V)
  src = V - 2
  snk = V - 1

  NM = N * M
  idx = lambda x, y : x * M + y
  for i in range(N) :
    for j in range(M) :
      if G[i][j] == 35 : continue
      for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)) :
        nx = i + dx
        ny = j + dy
        if not( 0 <= nx < N and 0 <= ny < M) : continue
        if G[nx][ny] == 35 : continue

        C.add_edge(NM+idx(i, j), idx(nx, ny), 1)

      C.add_edge(idx(i, j), NM+idx(i, j), 1) #out -> in
      if G[i][j] == 75 :
        KPos = (i, j)
        C.add_edge(src, idx(i, j), 10000)
        C.add_edge(idx(i, j), NM+idx(i, j), 10000)
      elif G[i][j] == 72 :
        HPos = (i, j)
        C.add_edge(idx(i, j), NM+idx(i, j), 10000)
        C.add_edge(NM+idx(i, j), snk, 10000)

  if not HPos or not KPos : return print(-1)
  for dx, dy in ((0, 1), (0, -1), (1, 0), (-1, 0)) :
    nx = KPos[0] + dx
    ny = KPos[1] + dy
    if not( 0 <= nx < N and 0 <= ny < M) : continue
    if G[nx][ny] == 72 : 
      return print(-1)
  
  print(C.calc(src, snk))
sol()

- 템플릿을 좀 더 잘 활용해야 할 것 같다.
- 인접벡터를 만드려는 순간 MLE가 나는것이 원인이였다. 인접리스트 템플릿을 쓰니 잘풀린다.